# 0. Cash Flow and Report

## 0.1 Problem

- **Main objective**: Generate consolidated reports from a trading extract, update cash flow and send email with reports.

- **Problem**: Read trade extract file, make reports about day, month and year operations, update cash flow and send email with attachments to a list of people, being possible to add 1 more attachment for each person.
 



## 0.2 Planning


**Algorithm steps**:

1. Import the necessary packages
2. Open the extract file as dataframe
3. Select columns and change type
4. Create new columns relevant to reporting
5. Generate reports
6. Create updated cash flow
7. Generate a spreadsheet with taxes payable on earned amounts, Separate by year, day trade or not, share, option and futures market by dividing rates for day trade and normal.
8. Read sender username and password
9. Ask if you want to add another file and verify that the files are in excel
10. Read recipient list and send email

**Tools**: Python 3.8.3 

**How?** email with attachments





# 1. Imports

In [1]:
import pandas as pd
import numpy as np
import datetime

import re

import smtplib

from email.message import EmailMessage

## 0.1. Helper Functions

# 2. Load data

In [2]:
def get_data(path):
    data = pd.read_excel('../database/ReportHistory.xlsx', header=5, engine='openpyxl')
    data.rename(columns=data.iloc[0], inplace= True)
    data = data.drop(labels=0, axis=0)
    return data



# 3.0 Data description

## 3.1 Rename columns

In [3]:
def change_dtypes(data):
    
    data = data[['Horário',    'Position',       'Ativo',        'Tipo',
            'Volume',    'Preço In',       'S / L',       'T / P',
       'Horário Out',   'Preço Out', 'Lucro']]
    
    # change dtypes
    data['Horário'] = pd.to_datetime(data['Horário'], format= '%Y-%m-%d ')
    data['Horário Out'] = pd.to_datetime(data['Horário Out'], format= '%Y-%m-%d ')

    data['Position'] = data['Position'].astype( np.int64)
    data['Preço In'] = data['Preço In'].astype( float )
    data['Preço Out'] = data['Preço Out'].astype( float )

    data['Lucro'] = data['Lucro'].astype( np.int64)
    
    return data



## 3.4 Feature extraction

In [4]:
def feature_extraction(df1):
    
    df1['Date'] = df1['Horário'].dt.normalize()
    df1['year'] =  df1['Horário'].dt.year
    df1['month'] = df1['Horário'].dt.month
    df1['day'] =   df1['Horário'].dt.day

    # Market classification

    df1['Market']= df1['Ativo'].apply(lambda x: 'futures' if x.startswith(('WIN', 'WDO')) else

                       'stock' if len(x) == 5 else 'options' )


    # day trade = 1 if day trade, 0 if not
    df1['day_trade']= df1.apply(lambda x: 1 if x['day'] == x['Horário Out'].day else 0, axis=1 )
    
    return df1



In [5]:
def financial_reports(df1):
    

    df1['yesterday'] = today - datetime.timedelta(days=1)

    yesterday_report = df1.loc[df1['Date'] == df1['yesterday'],['Horário',    'Position',       'Ativo',        'Tipo',
                'Volume',  'Lucro', 'Market', 'day_trade']]


    

    ## Save Report
    #yesterday_report.to_excel('reports/report{}.xls'.format(y))
    yesterday_report
    
    
    
    # Group by Date nr of trades and sum of Profif/loss
    profit_loss = df1[['Date', 'Lucro']].groupby('Date').sum().reset_index()

    trade_nr = df1[['Date', 'Ativo']].groupby('Date').count().reset_index()

    #merge 
    aux = pd.merge( trade_nr,profit_loss, how='inner', on='Date')
    aux.columns =['Date', 'Nr of Trades', 'Profit/Loss']

    # Last 10 days - summary (to_excel)
    summary_10_last_days = aux.sort_values('Date', ascending = False).head(10)
    summary_10_last_days.to_excel('reports/last_10_days_update{}.xlsx'.format(y))
    
    
    #Summarize stock, deal and total profit/loss per date using agg. 
    total_report = df1.groupby(['Date', 'Ativo']).agg( Deals=('Ativo','count'),Profit_Loss=('Lucro','sum')
                                            ).sort_values('Date', ascending= False)

    total_report.to_excel('../reports/total_report_summary{}.xlsx'.format(y))
    
    
    return yesterday_report, total_report







## 3.4  Filtering Variables

In [6]:
def cash_flow(df1):
    
    #Filtering Variables
    daily_spend = df1.groupby('Date').agg({'Lucro':['count','sum']}).reset_index()
    daily_spend= daily_spend.rename(columns={"count": "Nr. Trades", "sum": "Profit/Loss"})
    
    #preparing DataFrame
    c_flow = pd.DataFrame()

    # start initial blance
    initial = 10000

    initial_balance=[]
    initial_balance.append(initial)


    #c_flow.values
    lucro = daily_spend['Lucro']['Profit/Loss'].values
    tax = abs(lucro*0.01)

    for i in range(daily_spend.shape[0] -1):
        initial = initial + lucro[i] - tax[i]
        initial_balance.append(initial)

    #initial_balance
    c_flow['Date'] = daily_spend['Date']
    c_flow['initial_balance'] = initial_balance
    c_flow['Lucro'] = daily_spend['Lucro']['Profit/Loss']
    c_flow['withholding_tax']= c_flow['Lucro'].apply(lambda x: abs(0.01*x) )
    c_flow['final_balance'] = c_flow['initial_balance'] + c_flow['Lucro']- c_flow['withholding_tax']

    c_flow.sort_values('Date', ascending = False)

    # Export 
    c_flow.sort_values('Date', ascending = False).to_excel('../reports/cashflow{}.xlsx'.format(y))
    
    return None




# 4.0 Cash Flow

## 4.1 Income Tax 

In [7]:
def income_tax(df1):
    
    ### Tax Rules
    # each trade = 1% withholding tax if profit > 0 
    # income tax is calculated by month = 20% * profit - withholdtax if it´s a day trade, normal trade 15%, if loss no tax

    df1['withholding_tax'] = df1['Lucro'].apply(lambda x: abs(0.01*x) if x > 0 else 0 )

    df1['Profit_minus_withholding_tax'] = df1.apply (lambda x: x['Lucro'] - x['withholding_tax'] if x['Lucro'] > 0 else x['Lucro'], axis =1 )



    aux_tax_0 =  df1.groupby(['year','month', 'Market', 'day_trade']).agg( Profit= ('Lucro', 'sum'),Withholding_tax=('withholding_tax','sum'),Deals=('Ativo','count'),Profit_minus_tax= ('Profit_minus_withholding_tax', 'sum')).reset_index()


    aux_tax_0['income_tax'] = aux_tax_0.apply(lambda x: 0.20 * x['Profit_minus_tax'] if (x['day_trade'] == 1) & (x['Profit_minus_tax'] > 0) else 0.15* x['Profit_minus_tax'] if   (x['day_trade'] == 0 )& (x['Profit_minus_tax'] > 0)   else 0, axis=1 )
    aux_tax_0['Profit_net'] = aux_tax_0.apply(lambda x: x['Profit_minus_tax'] - x['income_tax'], axis=1 )


    aux_tx_summary = aux_tax_0.groupby(['year','month', 'day_trade']).agg(Deals=('Deals','count'),Profit= ('Profit', 'sum'),Withholding_tax=('Withholding_tax','sum'), Income_tax = ('income_tax', 'sum'), Profit_net = ('Profit_net', 'sum') ).sort_values('month')

    aux_tx_by_mkt = aux_tax_0.groupby(['year','month', 'Market','day_trade']).agg(Deals=('Deals','count'),Profit= ('Profit', 'sum'),Withholding_tax=('Withholding_tax','sum'), Income_tax = ('income_tax', 'sum'), Profit_net = ('Profit_net', 'sum') ).sort_values('month')
    aux_tx_summary.to_excel('../reports/tx_per_month-updated{}.xlsx'.format(y))
    aux_tx_by_mkt.to_excel('../reports/tx_per_mkt-updated{}.xlsx'.format(y))
    
    return None




# 5.0  Send email

In [8]:
# capture email and password from sender

def capture(arquivo): 
    nome_arq = arquivo
    try:
        arq = open(nome_arq, 'r') # abre arquivo em modo de leitura
        conteudo = arq.readlines() # lê as linhas do aquivo e guarda em uma lista
        arq.close()

        for linha in conteudo:
            if "email" in linha:
                lista_str = linha.split('=')
                email   = lista_str[1].strip()
                

            if "senha" in linha:
                lista_str = linha.split('=')
                senha     = lista_str[1].strip()
                              
    
    except FileNotFoundError:
        print(f"Arquivo {nome_arq} não encontrado!")
        
    return email, senha

In [9]:
def files(arq, name ): 
    
    add_file= str(input('Add another file to {}´s report ( only excel, please!)? ( N to exit )'.format(name)))

    regex = '(.xls|.xlsx)$' 

    spair_letter = [l for l in add_file   ] 
    

    if len(spair_letter) < 4:
            add_file = add_file.upper()[0]
            
    else:
        
        if add_file != "N":

            try: 
                if bool(re.search( regex, add_file )) == True  and add_file not in arq: 
                    
                    arq.append(add_file)
                else: 
                    print('Please check the file path and name. ej:("./reports/file.xlsx") ')
                    
                    
            except Exception as e:
                print('Error! : ',e)
        
            
    return arq



    

In [10]:
def send_mail_with_excel( from_email_file, name, recipient_email, subject, files_xls):
    #capture email and password "FROM"
    email, senha = capture(from_email_file)
    
    msg = EmailMessage()
    msg['Subject'] = subject + y
    msg['From'] = email
    msg['To'] = recipient_email
    
    email_content = f'''

        <p>Dear {name}, </p>
        <p>Please find attached detailed cash flow and trade report.  </p>
        <br />
        <p> <small><strong> Yesterday´s trade´s information   </strong></small> </p>
            {yesterday_report.to_html()}
        <br />
        <p>Taiz Alves</p>
        <p><small><strong>Data analyst </strong></small></p> '''

    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(email_content)
    
    
    ## add more than 1 file and ask 
    
    for excel_file in files_xls:
        with open(excel_file, 'rb') as f:
            file_data = f.read()
            msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename=excel_file)
    
    
    try: 
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(email, senha)
            smtp.sendmail(msg['From'], [msg['To']],msg.as_string().encode('utf-8'))
            smtp.quit()
            print('Email sent to {}'.format(recipient_email))
    except Exception as e:
        print('Error! : ',e)
        
    return None
        


## 5.0  if __name__ == '__main__':

In [11]:
# call all def

# get data
data = get_data('../database/ReportHistory.xlsx')


##transformation
data = change_dtypes(data)

print( 'Number of Rows: {}'.format( data.shape[0] ) )
print( 'Number of Cols: {}'.format( data.shape[1] ) )

print(data.dtypes)

#transformation
data = feature_extraction(data)
print(data.isna().sum() )
print('')
print(data.head() )

today = data['Date'].max()

#y =datetime.date.today() - datetime.timedelta(days=1)
y =today - datetime.timedelta(days=1)
y = y.strftime('%Y_%m_%d')


#reports
yesterday_report, total_report= financial_reports(data)

#cash flow
cash_flow(data)

income_tax(data)


<ipython-input-3-f4d6d1cea6c2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Horário'] = pd.to_datetime(data['Horário'], format= '%Y-%m-%d ')
<ipython-input-3-f4d6d1cea6c2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Horário Out'] = pd.to_datetime(data['Horário Out'], format= '%Y-%m-%d ')
<ipython-input-3-f4d6d1cea6c2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

Number of Rows: 332
Number of Cols: 11
Horário        datetime64[ns]
Position                int64
Ativo                  object
Tipo                   object
Volume                 object
Preço In              float64
S / L                  object
T / P                  object
Horário Out    datetime64[ns]
Preço Out             float64
Lucro                   int64
dtype: object
Horário         0
Position        0
Ativo           0
Tipo            0
Volume          0
Preço In       38
S / L          42
T / P          45
Horário Out     0
Preço Out      38
Lucro           0
Date            0
year            0
month           0
day             0
Market          0
day_trade       0
dtype: int64

              Horário   Position   Ativo Tipo Volume  Preço In   S / L  \
1 2020-10-01 14:00:00  477192759  WDOX20  buy      1    5645.5    5609   
2 2020-10-01 16:36:19  477549686  WDOX20  buy      1    5661.0  5620.5   
3 2020-10-02 12:00:00  478175765  WDOX20  buy      1    5643.0    5605   
4

In [2]:
## get receiver´s emails and nome 

emails = pd.read_excel("../email.xls")
names = [ n for n in emails.Name]
emails = [ e for e in emails.email]


for i in range (len(emails)): 
    arquivo = ['../reports/cashflow2021_03_28.xlsx' ,'../reports/tx_per_month-updated2021_03_28.xlsx']

    send_mail_with_excel('../email_senha.txt' , names[i], emails[i],  'financial report', files(arquivo, names[i]) )


['Taiz', 'Augusto']